In [3]:
!pip install optuna -q

In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

import optuna

In [5]:
X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values
X_train_mat100 = pd.read_csv('../data/Xtr_mat100.csv',sep=' ',header=None).values


# X_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0).values
# X_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0).values

In [6]:
print('x_train: {} y_train {}'.format(X_train_mat100.shape,y.shape))
# print('x_train: {} y_train {}'.format(X_train.shape,y.shape))
# print('x_test: {}'.format(X_test.shape))

x_train: (2000, 100) y_train (2000, 1)


In [7]:
X_train_mat100 = scale(X_train_mat100)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_mat100, y, test_size=0.1, random_state=42)

print(X_train.shape,X_val.shape,y_train.shape, y_val.shape)

(1800, 100) (200, 100) (1800, 1) (200, 1)


### Model Test

In [8]:
class logisticregression():
    def __init__(self,train_data,train_labels,lamda=0.2,lr=0.01,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        self._lambda = lamda
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))+(2*self._lambda*self.params)
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if self.print_every:
                if i%self.print_every == 0 or i == self.epoch-1:
                    print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result > 0.5 ] = 1
        result[result <= 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [9]:
def cross_validate(x_data,y_data,lr,lamda=0.2,epoch=10,k=5):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
        logistic = logisticregression(x_train,y_train,lamda=lamda,lr=lr,epoch=epoch,print_every=None)
        logistic.train()
        
        result = logistic.evaluate(x_test,y_test)
        aggrigate_result.append(result)
        
        value = sum(aggrigate_result)/len(aggrigate_result)
    return value if value!= None else 0

In [30]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    lamda = trial.suggest_loguniform('lamda', 0.2, 0.5)
    k =  trial.suggest_categorical('k', [4,5,8,10])
    epoch =  trial.suggest_int('epoch', 3, 15)
    return cross_validate(X_train_mat100, y,lr=lr,lamda=lamda,k=k,epoch=epoch)

In [31]:
# cross_validate(X_train_mat100, y,0.001,10)

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=200,show_progress_bar=True)

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in multiply



[I 2020-05-24 21:35:01,689] Finished trial#0 with value: 0.5275000000000001 with parameters: {'lr': 0.046664603171990185, 'lamda': 0.24826405591012668, 'k': 4, 'epoch': 7}. Best is trial#0 with value: 0.5275000000000001.
[I 2020-05-24 21:35:01,783] Finished trial#1 with value: 0.5685 with parameters: {'lr': 0.0028874376567811105, 'lamda': 0.4169542815556156, 'k': 8, 'epoch': 12}. Best is trial#1 with value: 0.5685.
[I 2020-05-24 21:35:01,894] Finished trial#2 with value: 0.524 with parameters: {'lr': 0.006916338839050143, 'lamda': 0.37727559560700274, 'k': 10, 'epoch': 14}. Best is trial#1 with value: 0.5685.
[I 2020-05-24 21:35:01,991] Finished trial#3 with value: 0.548 with parameters: {'lr': 0.007633427748203205, 'lamda': 0.37021795018362874, 'k': 4, 'epoch': 6}. Best is trial#1 with value: 0.5685.
[I 2020-05-24 21:35:02,092] Finished trial#4 with value: 0.5455000000000001 with parameters: {'lr': 0.08579812690307, 'lamda': 0.20399915773682092, 'k': 8, 'epoch': 13}. Best is trial#1 w

In [32]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)

df.sort_values(by=['value'])

,number,value,duration,params_epoch,params_k,params_lamda,params_lr
44,44,0.5225,00:00:00.089006,3,5,0.266792,0.035819
2,2,0.5240,00:00:00.109103,14,10,0.377276,0.006916
150,150,0.5270,00:00:00.083613,5,5,0.273965,0.029439
0,0,0.5275,00:00:00.116906,7,4,0.248264,0.046665
38,38,0.5380,00:00:00.095211,5,8,0.432876,0.007275
...,...,...,...,...,...,...,...
103,103,0.5950,00:00:00.085257,4,5,0.233263,0.000167
134,134,0.5955,00:00:00.086828,4,5,0.234450,0.000179
139,139,0.5960,00:00:00.084848,3,5,0.216162,0.000257
169,169,0.5960,00:00:00.095087,5,5,0.243923,0.000135


In [33]:
cross_validate(X_train_mat100,y,lamda=0.362124,lr=0.000254,k=5,epoch=15)

0.5930000000000001

In [34]:
# X_train, X_test, y_train, y_test
logistic = logisticregression(X_train,y_train,lamda=0.362124,epoch=15,print_every=1,lr=0.000254)
logistic.train()
        
print(logistic.evaluate(X_train,y_train))
print(logistic.evaluate(X_val,y_val))

Epoch : 0  Loss: 0.6931471805599452
Epoch : 1  Loss: 0.6727179950537673
Epoch : 2  Loss: 0.6598182886049019
Epoch : 3  Loss: 0.6510545320971816
Epoch : 4  Loss: 0.6448284553516899
Epoch : 5  Loss: 0.6402285314019652
Epoch : 6  Loss: 0.636715013723191
Epoch : 7  Loss: 0.6339553905553064
Epoch : 8  Loss: 0.6317367805999382
Epoch : 9  Loss: 0.6299179584471648
Epoch : 10  Loss: 0.6284021802839714
Epoch : 11  Loss: 0.6271212492262762
Epoch : 12  Loss: 0.6260258544267582
Epoch : 13  Loss: 0.625079528073846
Epoch : 14  Loss: 0.6242547549724377
0.6388888888888888
0.61


In [35]:
X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values

X_test_mat100[0,:]

X_test_mat100 = scale(X_test_mat100)

In [36]:

sumbission = []
for i in range(len(X_test_mat100)):
    result = logistic.predict(X_test_mat100[i])
    sumbission.append([i,int(result)])
    result

In [37]:
# sumbission
df = pd.DataFrame(sumbission)
df.columns = ['Id','Bound']
df.to_csv('test.csv',index=False)

In [38]:
df.head(50)

,Id,Bound
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,0
7,7,1
8,8,1
9,9,0
